In [1]:
!pip install faker PyMySQL tqdm

In [3]:
from faker import Faker
import pymysql
from tqdm import tqdm
import random
from datetime import datetime, timedelta

In [5]:
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="MySQL",
    database="dealership",
    autocommit=False,
    cursorclass=pymysql.cursors.Cursor
)

cursor = connection.cursor()
fake = Faker()

In [7]:
def random_date(start_year=2015, end_year=2024):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    delta = end - start
    return start + timedelta(days=random.randint(0, delta.days))

In [9]:
roles = ["USER", "ADMIN", "MODERATOR"]

for r in roles:
    cursor.execute("INSERT INTO roles (name) VALUES (%s)", (r,))

connection.commit()
print("Inserted roles.")

Inserted roles.


In [11]:
NUM_USERS = 1000
user_ids = []

for _ in tqdm(range(NUM_USERS)):
    username = fake.unique.user_name()
    email = fake.unique.email()
    password = fake.password()
    phone = fake.phone_number()
    enabled = random.choice([0, 1])
    role_id = random.randint(1, len(roles))

    cursor.execute("""
        INSERT INTO users (username, email, password, phone, enabled, role_id)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (username, email, password, phone, enabled, role_id))

    user_ids.append(cursor.lastrowid)

connection.commit()
print("Inserted users:", len(user_ids))

100%|██████████| 1000/1000 [00:01<00:00, 721.03it/s]

Inserted users: 1000


In [13]:
NUM_CARS = 1000
car_ids = []

brands_models = {
    "Toyota": ["Camry", "Corolla", "Rav4"],
    "Honda": ["Civic", "Accord", "CR-V"],
    "BMW": ["X5", "3 Series", "5 Series"],
    "Mercedes": ["C-Class", "E-Class", "GLA"],
    "Ford": ["Focus", "Fusion", "Mustang"]
}

for _ in tqdm(range(NUM_CARS)):
    brand = random.choice(list(brands_models.keys()))
    model = random.choice(brands_models[brand])
    year = random.randint(1995, 2024)
    mileage = random.randint(0, 300000)
    color = fake.color_name()
    price = round(random.uniform(5000, 100000), 2)
    description = fake.text(200)
    is_available = random.choice([0, 1])
    created_at = random_date()
    last_modified_at = random_date()
    user_id = random.choice(user_ids)

    cursor.execute("""
        INSERT INTO cars (
            brand, model, year, mileage, color, price, description,
            is_available, created_at, last_modified_at, user_id
        )
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (brand, model, year, mileage, color, price, description,
          is_available, created_at, last_modified_at, user_id))

    car_ids.append(cursor.lastrowid)

connection.commit()
print("Inserted cars:", len(car_ids))

100%|██████████| 1000/1000 [00:00<00:00, 1010.57it/s]

Inserted cars: 1000


In [15]:
NUM_EVENTS = 1000
event_ids = []

for _ in tqdm(range(NUM_EVENTS)):
    cursor.execute("INSERT INTO events () VALUES ()")
    event_ids.append(cursor.lastrowid)

connection.commit()
print("Inserted events:", len(event_ids))

100%|██████████| 1000/1000 [00:00<00:00, 2378.85it/s]

Inserted events: 1000


In [9]:
NUM_PHOTOS = 1000
photo_ids = []

# Get all existing car IDs
cursor.execute("SELECT id FROM cars")
car_ids = [row[0] for row in cursor.fetchall()]  # simple tuple access since using Cursor

if not car_ids:
    raise ValueError("No cars found! Populate cars table first.")

for _ in tqdm(range(NUM_PHOTOS)):
    file_name = f"{fake.uuid4()}.jpg"
    mime_type = "image/jpeg"
    data = fake.binary(length=128)  # very small blob
    upload_time = random_date().strftime('%Y-%m-%d %H:%M:%S')
    car_id = random.choice(car_ids)
    event_id = None  # not linking to events

    cursor.execute("""
        INSERT INTO photos (file_name, mime_type, data, upload_time, car_id, event_id)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (file_name, mime_type, data, upload_time, car_id, event_id))

    photo_ids.append(cursor.lastrowid)

connection.commit()
print("Inserted photos:", len(photo_ids))

100%|██████████| 1000/1000 [00:00<00:00, 1310.25it/s]

Inserted photos: 1000


In [ ]:
NUM_MESSAGES = 1000
message_ids = []

cursor.execute("SELECT id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]

for _ in tqdm(range(NUM_MESSAGES)):
    username = fake.user_name()
    phone = fake.phone_number()
    message = fake.text(300)
    created_at = datetime.now()

    if random.random() < 0.5:
        user_id = random.choice(user_ids)
    else:
        user_id = None

    cursor.execute("""
        INSERT INTO contact_messages (username, phone, message, created_at, user_id)
        VALUES (%s,%s,%s,%s,%s)
    """, (username, phone, message, created_at, user_id))

    message_ids.append(cursor.lastrowid)

connection.commit()
print("Inserted contact messages:", len(message_ids))


  0%|          | 0/1000 [00:00<?, ?it/s]


NameError: name 'user_ids' is not defined